# Generating SD files

Script that generates the median standard deviation from gradient files that have already been calculated; updating here to first linearly detrend the gradient dataset before calculating

In [1]:
%reset -f

In [12]:
# importing required libraries
import numpy as np
import xarray as xr
import pandas as pd
import os

from GradientProjectFunctions import MakeChangeDir
from GradTrendClasses import CalcPeriodSD

In [4]:
# setting up directories
dirGrad = '/home/hbyrne/Research/Gradient_project/gradient_project/Outputs/Gradients_NonParent_All'
dirSD = '/home/hbyrne/Research/Gradient_project/gradient_project/Outputs/SD_detrended'

In [3]:
models = ['CanESM5', 'MIROC6', 'GISS-E2-1-G', 'MPI-ESM1-2-LR', 'IPSL-CM6A-LR', 'MIROC-ES2L', 'ACCESS-ESM1-5', 'CNRM-CM6-1', 'CESM2', 'CNRM-ESM2-1', 'INM-CM5-0', 'UKESM1-0-LL', 'MPI-ESM1-2-HR']

In [ ]:
for model in models:

    # reading in the gradient files
    os.chdir(dirGrad)
    gradFiles = os.listdir(dirGrad)
    grads = [xr.open_dataarray(gradFile) for gradFile in gradFiles if model in gradFile]

    # iterate through this list and calculate SD for all periods for each file; save each file
    counter = 1
    for grad in grads:

        # detrending
        arrayTime = np.arange(0, len(grad.time))

        # fit the line
        slope, intercept = np.polyfit(arrayTime, grad, 1)

        # detrend
        yFit = intercept + slope*arrayTime
        gradDetrended = grad - yFit
        
        # calculate SD for every time period
        dfGradSD = CalcPeriodSD(gradDetrended, 20).periodSDDf

        # save it out to the directory
        MakeChangeDir(dirSD)
        fileName = model + '_' + str(counter) + '.csv'
        dfGradSD.to_csv(fileName, index = True)

        # print progress 
        print(f'{model}: {counter} / {len(grads)}')
        counter += 1